<a href="https://colab.research.google.com/github/GaneshSelvaraj717/Ganesh-Selvaraj/blob/master/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd # to load dataset
import numpy as np
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords # to get collection of stopwords
from nltk.tokenize import word_tokenize
import string
import gensim
import matplotlib.pyplot as plt

# layers of the architecture
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Dropout
from keras.layers.embeddings import Embedding
from keras.layers import Bidirectional

from keras.preprocessing.text import Tokenizer # to encode text to int
from keras.preprocessing.sequence import pad_sequences # to do padding or truncating
from keras.models import Sequential   # the model

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# New Section

In [2]:
file = "/content/corona_nlp_train.csv"
corona_reviews = pd.read_csv(file, encoding = 'latin-1')

FileNotFoundError: ignored

In [ ]:
corona_reviews.head()

In [ ]:
corona_reviews.isnull().values.any()

In [ ]:
corona_reviews1 = corona_reviews.drop(columns = ['UserName', 'ScreenName', 'Location', 'TweetAt'], axis = 1)

In [3]:
corona_reviews1

NameError: ignored

In [4]:
corona_reviews1.isnull().values.any()

NameError: ignored

In [ ]:
corona_reviews1 = corona_reviews1.dropna().reset_index(drop=True)

In [5]:
corona_reviews1.isnull().values.any()

NameError: ignored

In [ ]:
# Data Preprocessing 
def preprocess_text(sen):
    
    sen = re.sub('<.*?>', ' ', sen) # remove html tag

    tokens = word_tokenize(sen)  # tokenizing words

    tokens = [w.lower() for w in tokens]    # lower case

    table = str.maketrans('', '', string.punctuation)  # remove punctuations
    stripped = [w.translate(table) for w in tokens]

    words = [word for word in stripped if word.isalpha()]  # remove non alphabet
    stop_words = set(stopwords.words('english'))

    words = [w for w in words if not w in stop_words]   # remove stop words
    words = [w for w in words if len(w) > 2]  # Ignore words less than 2
    
    return words

In [6]:
sentiment_lines = []
sentences = list(corona_reviews1['Sentiment'])

NameError: ignored

In [ ]:
for sen in sentences:
    # Call the preprocess_text function on each sentence of the review text 
    sentiment_lines.append(preprocess_text(sen))

In [7]:
# Check for the length of the preprocessed text
len(sentiment_lines)

0

In [8]:
print(sentiment_lines[1])

IndexError: ignored

In [ ]:
# Now let’s convert the sentiment from string to a binary form of 1 and 0, 
# where 1 is for ‘positive’ sentiment and 0 for ‘negative’.
y = corona_reviews1['Sentiment']
y = np.array(list(map(lambda x: 1 if x=="positive" else 0, y)))
print(y)
count = np.count_nonzero(y)
print(count)

In [9]:
EMBEDDING_DIM = 100

# Train word2vec model after preprocessing the reviews
model = gensim.models.Word2Vec(sentences=sentiment_lines, size=EMBEDDING_DIM, window=5, workers=4, min_count=1)

RuntimeError: ignored

In [ ]:
words = list(model.wv.vocab)
print('Vocabulary size: %d' % len(words))

In [10]:
# Save model
filename = "corona_embedding_word2vec.txt"
model.wv.save_word2vec_format(filename, binary=False)

NameError: ignored

In [ ]:
import os 

embeddings_index = {}
f = open(os.path.join('','corona_embedding_word2vec.txt'), encoding="utf-8")

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:])
    embeddings_index[word] = coefs
f.close()

In [ ]:
# Tokenizer class from the keras.preprocessing.text module creates a word-to-index integer dictionary
# Vectorize the text samples
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentiment_lines)
sequences = tokenizer.texts_to_sequences(sentiment_lines)

In [11]:
# Pad sequences
word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

# Only consider the first  100 words of each movie review
max_length = 100

review_pad = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')
sentiment = y

print('Shape of pad tensor:', review_pad.shape)
print('Shape of sentiment tensor', sentiment.shape)

NameError: ignored

In [12]:
print(word_index)

NameError: ignored

In [13]:
# Padded reviews
print(review_pad)

NameError: ignored

In [14]:
# Adding 1 because of reversed 0 index
vocab_size = len(word_index) + 1

# Create a weight matrix for words in the training data
embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))

for word, index in word_index.items():
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    # If words not found in embedding matrix will be all 0's
    embedding_matrix[index, :] = embedding_vector

NameError: ignored

In [15]:
import os 

embeddings_index = {}
f = open(os.path.join('','corona_embedding_word2vec.txt'), encoding="utf-8")

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:])
    embeddings_index[word] = coefs
f.close()

FileNotFoundError: ignored

In [16]:
# Tokenizer class from the keras.preprocessing.text module creates a word-to-index integer dictionary
# Vectorize the text samples
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentiment_lines)
sequences = tokenizer.texts_to_sequences(sentiment_lines)

In [40]:
# Pad sequences
word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

# Only consider the first  100 words of each movie review
max_length = 100

review_pad = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')
sentiment = y

print('Shape of pad tensor:', review_pad.shape)
print('Shape of sentiment tensor', sentiment.shape)

Found 4 unique tokens
Shape of pad tensor: (28696, 100)
Shape of sentiment tensor (28696,)


In [41]:
print(word_index)

{'positive': 1, 'negative': 2, 'extremely': 3, 'neutral': 4}


In [42]:
print(review_pad)

[[1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [4 0 0 ... 0 0 0]
 ...
 [2 0 0 ... 0 0 0]
 [3 1 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]


In [43]:
# Adding 1 because of reversed 0 index
vocab_size = len(word_index) + 1

# Create a weight matrix for words in the training data
embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))

for word, index in word_index.items():
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    # If words not found in embedding matrix will be all 0's
    embedding_matrix[index, :] = embedding_vector

In [64]:
indices = np.arange(review_pad.shape[0])
print(indices)

review_pad = review_pad[indices]
sentiment = sentiment[  indices]
print(sentiment)
print(review_pad.shape[0])
num_test_samples = int(review_pad.shape[0])
print(num_test_samples)

X_train_pad = review_pad[:-num_test_samples]
y_train = sentiment[:-num_test_samples]
X_test_pad = review_pad[-num_test_samples:]
y_test = sentiment[-num_test_samples:]

[    0     1     2 ... 28693 28694 28695]
[0 0 0 ... 0 0 0]
28696
28696


In [55]:
X_train_pad.shape, y_train.shape, X_test_pad.shape, y_test.shape

((0, 100), (0,), (28696, 100), (28696,))

In [66]:
X_train_pad, y_train, X_test_pad.shape, y_test.shape

(array([], shape=(0, 100), dtype=int32),
 array([], dtype=int64),
 (28696, 100),
 (28696,))

In [56]:
model2 = Sequential()

# Define Model
embedding_layer = Embedding(vocab_size,
                            EMBEDDING_DIM, 
                            weights = [embedding_matrix],
                            input_length = max_length,
                            trainable=False)
model2.add(embedding_layer)
model2.add(Bidirectional(LSTM(64, recurrent_dropout=0.6)))
model2.add(Dropout(0.5))
model2.add(Dense(1, activation='sigmoid'))

model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 

In [57]:
print('Summary of the built model...')
print(model2.summary())

Summary of the built model...
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 100, 100)          500       
                                                                 
 bidirectional_3 (Bidirectio  (None, 128)              84480     
 nal)                                                            
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 85,109
Trainable params: 84,609
Non-trainable params: 500
_________________________________________________________________
None


In [61]:
history2 = model2.fit(X_train_pad, y_train, batch_size=128, verbose=1, epochs=5, validation_split=0.2)

ValueError: ignored